# MLflow K-serve example integration
This example shows how to train a regressor with scikit. Store the model object in the MLflow and deploy the same object with the K-serve. This example expects the Kubeflow and the MLflow bundle being deployed. Please reffer to [this guide](https://documentation.ubuntu.com/charmed-mlflow/en/latest/tutorial/mlflow-kubeflow/). Following environment variables are expected to be set (in case of charmed kubeflow they are set for you):

* MLFLOW_S3_ENDPOINT_URL: endpoint for object storage 
* MLFLOW_TRACKING_URI: endpoint for mlflow server
* AWS_SECRET_ACCESS_KEY: secret key for object storage
* AWS_ACCESS_KEY_ID: username for object storage

In [2]:
!printenv | grep AWS

AWS_ACCESS_KEY_ID=minio
AWS_SECRET_ACCESS_KEY=08MND4UU8TAW48U0TIZMN10N994YNU


In [3]:
!printenv | grep MLFLOW

MLFLOW_TRACKING_URI=http://10.1.137.14:5000/
MLFLOW_S3_ENDPOINT_URL=http://10.1.137.21:9000/


In [4]:
# first install necessary libs
!pip install scikit-learn mlflow==2.1.1 boto3


[notice] A new release of pip available: 22.3.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


First we train and store the model object to MLflow registry.

In [5]:
# The data set used in this example is from http://archive.ics.uci.edu/ml/datasets/Wine+Quality
# P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
# Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

import os
import warnings
import sys

import boto3
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    try:
        data = pd.read_csv(csv_url, sep=";")
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s",
            e,
        )

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    alpha = 0.5
    l1_ratio = 0.5

    # create bucket
    object_storage = boto3.client(
        "s3",
        endpoint_url=os.getenv("MLFLOW_S3_ENDPOINT_URL"),
        config=boto3.session.Config(signature_version="s3v4"),
    )
    default_bucket_name = "mlflow"

    buckets_response = object_storage.list_buckets()
    result = [
        bucket
        for bucket in buckets_response["Buckets"]
        if bucket["Name"] == default_bucket_name
    ]
    if not result:
        object_storage.create_bucket(Bucket=default_bucket_name)

    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        result = mlflow.sklearn.log_model(
            lr, "model", registered_model_name="ElasticnetWineModel2"
        )
        model_uri = f"{mlflow.get_artifact_uri()}/{result.artifact_path}"


Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7931640229276851
  MAE: 0.6271946374319586
  R2: 0.10862644997792614


Successfully registered model 'ElasticnetWineModel2'.
2023/10/23 09:31:32 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetWineModel2, version 1
Created version '1' of model 'ElasticnetWineModel2'.


Now you can visit the mlflow UI to see the model being registered. MLflow ui is running on Nodeport 31380 (by default, you can always inspect with `kubectl get svc -n kubeflow`). If you are running this on VM you can tunnel the port of the nodeport to your localhost with `ssh -i <keypair> -L 31380:localhost:31380 <user>@vm-ip>`. Afther that you can visit http://localhost:31380/.

You can also inspect the s3 object store, that it has the model with the codeblock bellow.

In [6]:
# Print list of files in the default bucket `mlflow`
response = object_storage.list_objects_v2(Bucket=default_bucket_name)
files = response.get("Contents")
for file in files:
    print(f"file_name: {file['Key']}, size: {file['Size']}")

file_name: 0/d65b584e828f45e18c79a7136455f67a/artifacts/model/MLmodel, size: 501
file_name: 0/d65b584e828f45e18c79a7136455f67a/artifacts/model/conda.yaml, size: 175
file_name: 0/d65b584e828f45e18c79a7136455f67a/artifacts/model/model.pkl, size: 878
file_name: 0/d65b584e828f45e18c79a7136455f67a/artifacts/model/python_env.yaml, size: 121
file_name: 0/d65b584e828f45e18c79a7136455f67a/artifacts/model/requirements.txt, size: 67


Now the environment variable `model_uri` holds the value of location of the trained model in the MinIO artifact store.

In [7]:
# This function deploys the Model stored in Mlfow directly to Cluster with kserve deployment
def deploy(model_uri:str = "default_model_uri", user_namespace:str = "user123"):
    import subprocess
    
    with open("/tmp/manifest.yaml", "w") as f:
        manifest = """
apiVersion: "serving.kserve.io/v1beta1"
kind: "InferenceService"
metadata:
  name: "wine-regressor"
spec:
  predictor:
    serviceAccountName: kserve-controller-s3
    sklearn:
      storageUri: """+model_uri+"""
        """
        f.write(manifest)
    
    # Example of how to talk directly to kubeapi
    result = subprocess.call(['kubectl', 'apply', '-f', '/tmp/manifest.yaml', '-n', user_namespace])
    assert result == 0

In [8]:
deploy(model_uri, "user123")

inferenceservice.serving.kserve.io/wine-regressor created


Now we need to wait until the Inference service is active to test the prediction endpoint.

In [12]:
# This call assumes you port-forwarded the service of the predictor 
# kubectl port-forward -n user123 svc/wine-regressor-predictor 8889:80
!curl  -s http://localhost:8889/v1/models/wine-regressor:predict \
  -H "Content-Type: application/json" \
  -d '{"instances":[[10.1, 0.37, 0.34, 2.4, 0.085, 5.0, 17.0, 0.99683, 3.17, 0.65, 10.6]]}'

{"predictions":[5.731344540042413]}